<a href="https://colab.research.google.com/github/Mahenaz17595/EasyLeaveManagementSystem/blob/master/Rossmann_Using_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import Statements**

In [0]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2


In [2]:
!pip install fastai==0.7.0

In [10]:
from fastai.imports import *
from fastai.structured import *
from fastai.column_data import *
import sklearn.impute.SimpleImputer
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics
import altair as alt
from altair import Chart, X, Y, Axis, SortField
alt.renderers.enable('notebook')
from pandas import Timestamp
#import ggplot
np.set_printoptions(threshold=50, edgeitems=20)
from pdpbox import pdp
from plotnine import *
import feather
import pandas as pd
#import pdvega

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

ImportError: ignored

**Load Feather Data**

In [0]:
# File path
path_processed = 'data/processed/'
!ls {path_processed}

In [0]:
#Load feature engineered dataset using feather format
joined = feather.read_dataframe(f'{path_processed}joined')
joined_test = feather.read_dataframe(f'{path_processed}joined_test')
joined.columns

**Define Categorical & Continuous Variables**


* The dataset contains a mix of continuous and categorical variables.
* It is useful to explicitly identify and define categorical variables as pandas categories to:
* reduce the tree computation cost for RandomForests.
* create RandomForest model with one hot encoding.
* create embedding matrices for neural nets.

In [0]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
    'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
    'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw',
    'SchoolHoliday_fw', 'SchoolHoliday_bw']

contin_vars = ['CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']
n = len(joined); n

In [0]:
dep = 'Sales'
joined = joined[cat_vars+contin_vars+[dep, 'Date']].copy()

In [0]:
joined_test[dep] = 0
joined_test = joined_test[cat_vars+contin_vars+[dep, 'Date', 'Id']].copy()

**Convert Categorical Variables to Pandas Categories.**

Loop through the list of categoricals and convert them to Pandas categories.

In [0]:
for v in cat_vars: 
    joined[v] = joined[v].astype('category').cat.as_ordered()

The same category mapping applied on training data is applied to the test set.

Eg: If saturday is a 6 in the training set, apply_cats() ensures that same encoding is applied to the test set also.

In [0]:
apply_cats(joined_test, joined)


**Convert Continuous Variables to Float**

Loop through the list of continuous variables and set them as float (Neural Nets/PyTorch requirement).

In [0]:
for v in contin_vars:
    joined[v] = joined[v].fillna(0).astype('float32')
    joined_test[v] = joined_test[v].fillna(0).astype('float32')

In [0]:
# Set date as the index
joined_samp = joined.set_index("Date")

**Preprocessing for RandomForests**

* Replace categories with their numeric codes
* Handle missing values
* Assign the dependent variable as a separate variable

**Missing Values:**

* Replace missing values for categories with 0.
(Missing categorical variables are assigned as -1 by default. Hence all categorical variables are bumped up by 1, so the missing categorical variables are assigned 0, and the numeric codes for other categorical variables start from 1.)
* Replace missing values for continuous variables with their medians.
**Function proc_df( )**

**Parameters**

* joined_samp: Independent variables
* Sales: Dependent variable

In [0]:
df, y, nas = proc_df(joined_samp, 'Sales')
y = np.log(y)

In [0]:
joined_test = joined_test.set_index("Date")


In [0]:
df.isnull().sum().sort_values(ascending=False).head(7)


**Define Training Data and Validation Set**


In [0]:
def split_vals(a,n): 
    return a[:n], a[n:]

In [0]:
n_valid = 10000
n_trn = len(df)-n_valid
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

In [0]:
raw_train, raw_valid = split_vals(joined_samp, n_trn)


**Error definition : Root Mean Square Percent Error (RMSPE)**

* Root Mean Square Percent Error (RMSPE): Evaluation metric suggested by Kaggle in the Rossmann Competition.
* RMSPE means we are penalized based on the ratio of the predicted value and the actual value.
* Calculating the Root Mean Squared Error of the log(data) will give us the Root Mean Squared Percent Error.

In [0]:
def inv_y(a): return np.exp(a)

def exp_rmspe(y_pred, targ):
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))/targ
    return math.sqrt((pct_var**2).mean())
def print_score(m):
    res = [exp_rmspe(m.predict(X_train), y_train), exp_rmspe(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [0]:
**set_rf_samples() :**  * Randomly samples 50000 rows from the dataset each time.


# **RandomForest Model**

**Function RandomForestRegressor ( )**

**Parameters**

**n_estimators :** Number of trees.

**min_sample_leaf :** Depth of the tree.

**max_features :** Randomly sample columns at each split. Rationale:

The less correlated your trees are with each other, the better.
If one column was more predictive than all other columns then every single tree will be split based on that column. But there might be some interaction of variables where that interaction is more important than the individual column. So if every tree always splits on the same thing the first time, we will not get much variation in those trees.
Using Max features - we can carryout column sampling - i.e for every individual binary split, we choose from a different subset of columns. 0.5 means we randomly choose a half of them.

**n_jobs=-1**

Random forests are trivially parallelizable.
n_jobs=-1 tells the random forest regressor to create a separate job/process for each CPU core.

**Out-of-bag (OOB) Score :**

OOB score indcates the model error on the rows not included in the training set.
Its an additional metric to check if the model is overfitting.

# **Single Tree**

In [0]:
m = RandomForestRegressor(n_estimators=1, max_depth=3, bootstrap=False, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [0]:
draw_tree(m.estimators_[0], df, precision=3)


In [0]:
#set_rf_samples(50000)


**Function fit ( )**

**Parameters**

X_train: Independent Variables

y_train: Dependent Variable

In [0]:
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=2, max_features=0.99, n_jobs=-1,oob_score=True)
m.fit(X_train, y_train)

In [0]:
print_score(m)


**Feature Importance**

The feature importance indicates the most important columns.
Working:

**Step 1:** Train a random forest model (m).

**Step 2:** Randomly shuffle each column and use the model (m) to predict the dependent variable. Now that the coloumn is randomly shuffled, the relationship between the dependent variable and the independent variable is lost, resulting in a poor predictive accuracy. This difference enables us to calculate the feature importance.

**Function rf_feat_importance ( )**
**Parameters**

**m :** RandomForest model

**df :** dataframe

In [0]:
fi = rf_feat_importance(m, df); fi[:10]


In [0]:
fi.plot('cols', 'imp', figsize=(10,6), legend=False);


In [0]:
def plot_fi(fi): 
    return fi.plot('cols', 'imp', 'barh', figsize=(16,10), legend=False)


In [0]:
plot_fi(fi[:30]);


* **CompetitionDistance and Promotions turn out to be the most important features.**

# Fine Tune Model Based on Feature Importance
**Remove the colums whose feature importance is less than or equal to 0.01 and check if there is any any impact on model accuracy and feature importance.**

In [0]:
# Remove columns whose feature importance is less than 0.01
to_keep = fi[fi.imp>0.005].cols; len(to_keep)

In [0]:
df_keep = df[to_keep].copy()
X_train, X_valid = split_vals(df_keep, n_trn)

In [0]:
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.99,n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

**The R-Square accuarcy is very similar to the oriignal model, but the model is a bit simpler now.**


In [0]:
fi = rf_feat_importance(m, df_keep)
plot_fi(fi);

**The feature importance remains the same after eliminating less important variables.**


**One-hot encoding**
* Convert coulmuns with cardinality less than 7 into seperate binary columns.
* proc_df's optional max_n_cat argument will turn categorical variables into new columns.
* Now some of these new columns may prove to have more important features than in the earlier situation, where all categories were in one column.
* For example, the column StoreType which has 4 categories:

* a
* b
* c
* d

gets turned into 6 new columns:

* StoreType_a
* StoreType_b
* StoreType_c
* StoreType_d

and the column StoreType gets removed.

In [0]:
df_trn2, y_trn, nas = proc_df(joined_samp, 'Sales',max_n_cat=7)
y_trn = np.log(y_trn)
#df_trn2.isnull().sum().sort_values(ascending=False).head(7)

In [0]:
#df_trn2.columns.str.endswith('_nan', na=False)
#df_trn2[df_trn2.columns[pd.Series(df_trn2.columns).str.endswith('_nan')]].describe()

for c in df_trn2.columns:
    if c.endswith('_nan'):
        if c in df_trn2.columns: df_trn2.drop(c, inplace=True, axis=1)

In [0]:
df_trn2.columns


In [0]:
X_train, X_valid = split_vals(df_trn2, n_trn)
m_ohe = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.99, n_jobs=-1, oob_score=True)
m_ohe.fit(X_train, y_train)
print_score(m_ohe)

In [0]:
fi = rf_feat_importance(m_ohe, df_trn2)
plot_fi(fi[:25]);


**Data exploration Based on Feature Importance**

**Competition Distance**

The feature importance function indiactes the competition distance to be the most important variable.
Lets understand this variable better.

In [0]:
df_tmp= joined_samp.copy()
df_tmp = df_tmp.groupby('Store', as_index = False).agg({'Assortment': 'min','StoreType':'min',
                                                    'CompetitionDistance': 'max','CompetitionOpenSinceYear': 'max',
                                                    'CompetitionMonthsOpen':'max','Sales': 'mean'})

(ggplot(data=df_tmp,mapping=aes(x='CompetitionDistance')) 
+ geom_histogram(bins =15, colour="black")
+ labs(
        title ='Competition for most of the Rossmann stores is right skewed',
        x = 'Competition Distance',
        y = 'Store Count',
    ) 
 + geom_vline(xintercept = 2320, size = 1, colour = "#FF3721",linetype = "dashed")
+ theme_bw()
)

In [0]:
df_tmp.CompetitionDistance.median()


In [0]:
df_tmp[df_tmp.CompetitionDistance <= df_tmp.CompetitionDistance.median()].Store.count()


In [0]:
(ggplot(data=df_tmp,mapping=aes(x='CompetitionDistance', y = 'Sales')) 
+ geom_point()
+ labs(
        title ='Mean Sales for Rossmann Stores are higher with decreasing order of competition distance',
        x = 'Competition Distance',
        y = 'Mean Sales',
    ) 
+ geom_vline(xintercept = 2320, size = 1, colour = "#FF3721",linetype = "dashed")
)

**Partial Dependece Plots**

In [0]:
x = get_sample(X_train[X_train.CompetitionDistance <=3000], 1000)


In [0]:
def plot_pdp(feat, clusters=None, feat_name=None):
    feat_name = feat_name or feat
    p = pdp.pdp_isolate(m_ohe, x, x.columns, feat)
    return pdp.pdp_plot(p, feat_name, plot_lines=True,
                        cluster=clusters is not None,
                        n_cluster_centers=clusters)

In [0]:
plot_pdp('CompetitionDistance', clusters=10)


* **The Competition Distance < 500 impacts the sales of the store.**


**Promotions**

In [0]:
#df_promo = joined_samp.groupby('Promo',as_index = False).agg({'Sales': 'mean'})
df_promo_store = joined_samp.groupby(['Store','Promo'],as_index = False).agg({'CompetitionDistance': 'max','StoreType':'min','Assortment': 'min','Sales': 'mean'})
df_promo_store.head()

#df_tmp= joined_samp.copy()
#df_tmp = df_tmp.groupby('Store', as_index = False).agg({'Assortment': 'min','StoreType':'min',
#                                                    'CompetitionDistance': 'max','CompetitionOpenSinceYear': 'max',
#                                                    'CompetitionMonthsOpen':'max','Sales': 'mean'})

In [0]:
ggplot(df_promo_store, aes(x = 'CompetitionDistance', y ='Sales'))
 + geom_point()
 + geom_smooth(
        aes(x = 'CompetitionDistance',
            y = 'Sales'),
           colour='red')
 + labs(
        title ='Promotions have a considerable impact on the mean sales',
        x = 'Competition Distance',
        y = 'Sales (Mean)',
    )
+
facet_wrap('Promo')
)


In [0]:
(
ggplot(df_promo_store[df_promo_store.CompetitionDistance <=2320], aes(x = 'CompetitionDistance', 
                                                       y ='Sales'))
 + geom_point()
 + geom_smooth(
        aes(x = 'CompetitionDistance',
            y = 'Sales'),
           colour='red')
 + labs(
        title ='Promotions have a considerable impact on the mean sales',
        x = 'Competition Distance',
        y = 'Sales (Mean)',
    )
+
facet_wrap('Promo')
)


**Summary**

* RandomForest model gave us a decent accuracy for Sales prediction.
* The accuracy obtained would put us in the top 2000 of the public leaderboard.
* More importantly it helped us identify the important variables and help us understand the data better.
* However if we need to obtain better preduction results we need to use Neural networs with hidden layers to better capture the interactions.

#**Neural networks**

**To Run on a Random Subset of the Dataset**
Train the neural net on a small sample

* to ensure everything is working fine,
* to determine the optimum hyperparameters and architectures.

In [0]:
idxs = get_cv_idxs(n, val_pct=150000/n)
joined_samp = joined.iloc[idxs].set_index("Date")
samp_size = len(joined_samp); samp_size

**To Run on the Complete Dataset**


In [0]:
samp_size = n
joined_samp = joined.set_index("Date"); samp_size

In [0]:
joined_samp.head(2)


**Note:**

* The year variable is replaced by 3, and categorical variables are replaced by the pandas categories code in above output snippet.

* Important step for creating embedding matrices later.

**Define Training Data and Validation Set**
In time series data, cross-validation is not random.
Holdout data is generally the most recent data. Hence we take 25% of the rows sorted by date as the initial validation set. (https://www.fast.ai/2017/11/13/validation-sets/)

In [0]:
train_ratio = 0.75
# train_ratio = 0.9
train_size = int(samp_size * train_ratio); train_size
#val_idx = list(range(train_size, len(df)))

**Validation Set**

We need to predict the next two weeks of sales in the test set, therefore we create a validation set which is the last two weeks of our training set.

In [0]:
val_idx = np.flatnonzero(
    (df.index<=datetime.datetime(2014,9,17)) & (df.index>=datetime.datetime(2014,8,1)))

**To Run on Complete Dataset for the Final Model**

* We retrain the final model on the whole dataset including the validation set.
* Fastai -assumes we always have a validations set.
* Hence define the first row as a validation set as a workaround.

In [0]:
val_idx=[0]


**Create Model Data Object**

ColumnarModelData represents a training set, validation set and an optional test set of a standard structured data.

* path_processed: Specifies the location to store the model files.
* val_idx: List of indices of the rows of the validation set
* df: Data frame
* y: Dependent Variable
* cat_flds: List of columns to be treated as categorical variables.

In [0]:
md = ColumnarModelData.from_data_frame(path_processed, val_idx, df, y.astype(np.float32), cat_flds=cat_vars, bs=128,
                                       test_df=df_test)

**Embeddings**

In [0]:
cat_sz = [(c, len(joined_samp[c].cat.categories)+1) for c in cat_vars]


In [0]:
# Sttore: 1116 -> indicates the number of rows in the embedding matrix
cat_sz

* **Some columns have more categories or levels than others.**
* **Eg: Store has over 1000 levels.**

**Embedding Matrix**
* Embedding matrices have to shown to perform better than one-hot encodings.
* Embeddings - by having higher dimensionality vector rather than just a single number, gives the deep learning network a chance to learn rich representations.

**Embedding Matrix - Definition**
* Cardinality of the variables is used to define the size of the embedding matrix.
* Each level will be assosiated with a vector based on the length as per the rules defined below:
* Rule 1 - Dimensionality of the embeddings are set as half of the discrete value the category has, up to maximum of 50.
* Rule 2 - Even if there were no missing values set an extra one aside for unknown.

In [0]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]
# Returns a list of tuples

In [0]:
emb_szs


In [0]:
max_log_y = np.max(y)
y_range = (0, max_log_y*1.2)

**Function get_learner ( )**

**Parameters**

* emb_szs: Embedding size
* len(df.columns)-len(cat_vars): Number of continuous variables in the data frame.
* 1: Number of outputs (Output of the last linear layer - We need to predict one number i.e Sales)
* 0.04: Dropout of the embedding matrix.
* [1000,500]: Activations in the linear layers.
* [0.001,0.01]: Dropouts to use in subsequent layers

**Dropouts**

Output of each linear layer is going to be a rank 1 tensor.
Dropout is going to remove some of the activations.

In [0]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
m.summary()

In [0]:
lr = 1e-3
m.lr_find()

In [0]:
m.sched.plot(100)


In [0]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-3

**Function fit ( )**

**Parameters**

* lr : Learning Rate
* 2 : Number of Cycles
* metrics: Print the results of this function at the end of each cycle

In [0]:
m.fit(lr, 2, metrics=[exp_rmspe], cycle_len=4)


**Complete Dataset**

In [0]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-3

**Hyperparameter Tuning**

In [0]:
m.fit(lr, 3, metrics=[exp_rmspe])


In [0]:
m.fit(lr, 3, metrics=[exp_rmspe], cycle_len=1)


**Test Set**

In [0]:
m.save('val0')


In [0]:
m.load('val0')


In [0]:
x,y=m.predict_with_targs()


In [0]:
exp_rmspe(x,y)


# **Summary**
* Neural Networks gives really good accuracy beased on Root Mean Square Percent Error.
* It better captures the complex time series interction through the use of enrtity embedding matrices.
* RandomForests was useful for initial data exploration and understand the results of the neural nets.
